In [4]:
import json
import copy
import src.models.a2c as a2c
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed

import concurrent.futures
import numpy as np
import tensorflow as tf
import gym
import scipy.signal
from src.utils import *
import argparse
from time import sleep

import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

discount = 0.99
env = get_env("configs/env.json")

sds = []
cnt = []
with open("configs/env.json", 'r') as f:
    config = json.load(f)

observation = env.reset(seq_no=0 % 50)
discount = 1.00
max_job_length = config['max_job_length']
capa = config['n_resource_slot_capacities']
n_resources = len(config['n_resource_slot_capacities'])
input_size = max_job_length * n_resources + 1
state_size = np.prod(observation['machine'].shape)
embedding_size = hidden_size = 24

In [5]:
import importlib
import numpy as np 

In [9]:

from tqdm import tqdm
importlib.reload(a2c)

def get_trajs(model, env, seq_no=5):
    ep_buffer = []

    list_s, list_a, list_r, list_y = [], [], [], []
    s = env.reset(seq_no=seq_no)
    for ep_len in range(config['ep_force_stop']):
        action, _ = model.get_action(s)
        s2, r, done, info = env.step(action)
        list_s.append(s)
        list_a.append(action)
        list_r.append(r)
        if done:
            break
        s = s2
    return list_s, list_a, list_r

envs = []
for i in range(10):
    envs.append(get_env("configs/env.json"))
    
def __main__():
    #trajWorkers = [traj_worker(model) for _ in range(num_train_seq)]
    model = a2c.model(
        discount, state_size, input_size, embedding_size, hidden_size,
        max_job_length, capa)
    def call(model, env):
        sqno = np.random.randint(0, 100)
        obs, acts, rews = get_trajs(model, env, seq_no=sqno)
        al, cl = model.train(obs, rews, acts)
        return al, cl
    with ThreadPoolExecutor(max_workers=8) as ex:
        for iter in tqdm(range(1000)):
            fts = []
            for i in range(8):
                fts.append(ex.submit(call, model, envs[i]))
            for future in as_completed(fts):
                al, cl = future.result()
            if (iter % 5) == 0:
                rews = []
                slowdowns = []
                fjc = []
                usages = np.empty(shape=(0, 3))

                for i in range(30):
                    s = env.reset(seq_no=100 + i)
                    rr = []
                    for ep_len in range(config['ep_force_stop']):
                        action, _ = model.get_action(s)
                        usage = (100 - env.machine.avbl_slot[0]) / 100.0
                        #print(usage)
                        usages = np.vstack([usages, usage])

                        s2, r, done, info = env.step(action)
                        rr.append(r)
                        if done:
                            break
                        s = s2
                    rews.append(np.mean(rr))
                    fjc.append(finisihed_job_cnt(info))
                    slowdowns.append(get_avg_slowdown(info))
                print("avg rewards: %0.3f\tavg slowdowns: %0.3f  fjc: %d" %
                        (np.mean(rews), np.mean(slowdowns), np.sum(fjc)))
                print(np.mean(usages, axis=0))

if __name__ == "__main__":
    __main__()


  0%|          | 1/1000 [00:23<6:35:06, 23.73s/it]

avg rewards: -1.851	avg slowdowns: 5.788  fjc: 889
[0.40396125 0.44243631 0.42848583]


  1%|          | 6/1000 [01:45<5:00:33, 18.14s/it]

avg rewards: -1.823	avg slowdowns: 5.593  fjc: 889
[0.40791304 0.44676449 0.43267754]


  1%|          | 11/1000 [03:11<5:19:00, 19.35s/it]

...

 99%|█████████▉| 991/1000 [3:59:01<01:46, 11.81s/it]

avg rewards: -1.875	avg slowdowns: 5.817  fjc: 889
[0.40454186 0.44307222 0.42910169]


100%|█████████▉| 996/1000 [3:59:50<00:45, 11.34s/it]

avg rewards: -1.749	avg slowdowns: 5.483  fjc: 889
[0.40108301 0.43928393 0.42543285]


100%|██████████| 1000/1000 [4:00:29<00:00,  9.97s/it]
